<a href="https://colab.research.google.com/github/jdeiros/soa-2020/blob/master/HPC/Deiros_Jeronimo_ejercicio_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Introducción
Por lo general, las computadoras se utilizan para compilar y analizar los resultados de encuestas y estudios de opinión.

El siguiente cuaderno calcula la media y la moda de los N valores de los elementos de un vector (resultados), lo hace en forma secuencial y en tambien en paralelo utilizando la biblioteca omp en el lenguaje C, que corre en la plataforma Colab ejecutado desde codigo python. 

Cada elemento del vector se inicializa con resultados aleatorios de puntajes (numeros enteros entre 0 y 9).

El algoritmo para el calculo de la media y la moda se basa en un ejemplo práctico del libro "C/C++ Cómo Programar"[3]

Su objetivo es aprender a utilizar Python[2] en la plataforma Colab [1] y la programación secuencial y en paralelo con openMP.


#2. Armado de ambiente
No es necesario para este ejercicio.

#3. Desarrollo

#3.1 Creación en python del archivo que tiene el codigo C para la ejecución del programa encuesta.

In [104]:
# Codigo Python, que tiene el código C de la ejecución del programa encuesta.
code = """
#include <iostream>
#include <vector>
#include <cstdlib>
#include <sys/time.h>
#include <omp.h>

// ----------------------------------------------------------------------------
// Macros que miden el tiempo.

static double dHashTiempoHistory[3];
static struct timeval tv;

#define TIEMPO_INI( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = tv.tv_sec + tv.tv_usec/1000000.0;
   
   
#define TIEMPO_FIN( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = ((tv.tv_sec + tv.tv_usec/1000000.0) - dHashTiempoHistory[ h ]) * 1000; // Devuelvo en milisegundos
#define TIEMPO_GET( h ) dHashTiempoHistory[ h ]

#define HTH_TOTAL         1
#define HTH_MODA_SEC      2
#define HTH_MODA_OMP      3
#define RANGO_PUNTAJES    10
// ----------------------------------------------------------------------------

int main(int argc, char* argv[]) 
{ 
	int i,c;
	TIEMPO_INI( HTH_TOTAL )

	// Leo los parametros.
	if( argc != 2 )
	{
		std::cerr<< " Error en los parametros de indicar: (Tamanio vector resultados)."<<argc<<std::endl;
		exit( -1 );
	}

	int cantidad_elementos = atoi( argv[1] );	
	double sumatoria_secuencial = 0;
	double sumatoria_paralelo = 0;
	// --------------------------------------------
	// Defino la memoria de los vectores.
	std::vector<int> resultados(cantidad_elementos);
	std::vector<int> frecuencias_secuencial( RANGO_PUNTAJES );
	std::vector<int> frecuencias_paralelo( RANGO_PUNTAJES );

	//Inicializo en cero los vectores de frecuencias
	for(int i=0; i < frecuencias_secuencial.size(); i++)
		frecuencias_secuencial[i]=0;
	for(int i=0; i < frecuencias_paralelo.size(); i++)
		frecuencias_paralelo[i]=0;

	//Cargo los puntajes aleatorios (0 a 9) en cada elemento del vector resultados.
	srand ( time(NULL) );
	for (int i=0;i<resultados.size();i++)
		resultados[i] = (int)(rand()%RANGO_PUNTAJES);

	// --------------------------------------------
	// Realizo el calculo de frecuencias y la sumatoria en forma secuencial.

	TIEMPO_INI( HTH_MODA_SEC )
	for(i=0;i < resultados.size(); i++)
	{
		sumatoria_secuencial += resultados[i];
		++frecuencias_secuencial[resultados[i]];
	}
	TIEMPO_FIN( HTH_MODA_SEC )

	// --------------------------------------------
	// Realizo el calculo de frecuencias y la sumatoria con OMP.

	TIEMPO_INI( HTH_MODA_OMP )

	#pragma omp parallel for
	for(i=0;i<resultados.size();i++)
	{
		sumatoria_paralelo += resultados[i];
		++frecuencias_paralelo[resultados[i]];
	}
	TIEMPO_FIN( HTH_MODA_OMP )
	
	
	// Obtengo la moda del vector_frecuencias obtenido secuencialmente
	int mas_grande_secuencial = 0;
	int valor_moda_secuencial = 0;
	for(i=0; i < frecuencias_secuencial.size(); i++)
	{
		if(frecuencias_secuencial[i] > mas_grande_secuencial)
		{
			mas_grande_secuencial += frecuencias_secuencial[i];
			valor_moda_secuencial = i;
		}
	}
	
	// Obtengo la moda del vector_frecuencias obtenido en forma paralela con omp
	int mas_grande_paralelo = 0;
	int valor_moda_paralelo = 0;
	for(i=0; i < frecuencias_paralelo.size(); i++)
	{
		if(frecuencias_paralelo[i] > mas_grande_paralelo)
		{
			mas_grande_paralelo += frecuencias_paralelo[i];
			valor_moda_paralelo = i;
		}
	}
	
	// --------------------------------------------
	// Muestro los resultados.
	std::cout<< "****************************************************************************************" <<std::endl;
	
	std::cout<<"resultados : ["; 
	for(i=0;i<resultados.size();i++)
	{
		std::cout<<resultados[i]<< ", ";
	}
	std::cout<<"]"<<std::endl; 

	std::cout<<"frecuencias_secuencial : [";
	for(i=0;i<frecuencias_secuencial.size();i++)
	{
		std::cout<<frecuencias_secuencial[i]<< ", ";
	}
	std::cout<<"]"<<std::endl; 

	std::cout<<"frecuencias_paralelo : [";
	for(i=0;i<frecuencias_paralelo.size();i++)
	{
		std::cout<<frecuencias_paralelo[i] << ", ";
	}
	std::cout<<"]"<<std::endl; 

	std::cout<<"sumatoria_secuencial: "<<sumatoria_secuencial<<std::endl;
	std::cout<<"media_secuencial: "<<sumatoria_secuencial/cantidad_elementos<<std::endl;
	std::cout<<"sumatoria_paralelo: "<<sumatoria_paralelo<<std::endl; 
	std::cout<<"media_paralelo: "<<sumatoria_paralelo/cantidad_elementos<<std::endl;
	
	std::cout<<"mas_grande_secuencial: "<<mas_grande_secuencial<<std::endl; 
	std::cout<<"valor_moda_secuencial: "<<valor_moda_secuencial<<std::endl;
	std::cout<<"mas_grande_paralelo: "<<mas_grande_paralelo<<std::endl; 
	std::cout<<"valor_moda_paralelo: "<<valor_moda_paralelo<<std::endl;

	TIEMPO_FIN( HTH_TOTAL )
	
	std::cout<<"****************************************************************************************" <<std::endl;
	std::cout<<"Valores Reales  :" <<std::endl;
	std::cout<<"Tiempo TOTAL     : "<<TIEMPO_GET(HTH_TOTAL   )<<" [ms]"<<std::endl;
	std::cout<<"Tiempo axpy Sec  : "<<TIEMPO_GET(HTH_MODA_SEC)<<" [ms]"<<std::endl;
	std::cout<<"Tiempo axpy Omp  : "<<TIEMPO_GET(HTH_MODA_OMP)<<" [ms]"<<std::endl;
	std::cout<<std::endl;
	std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_MODA_SEC)<<" / "<<TIEMPO_GET(HTH_MODA_OMP)<<" = "<<TIEMPO_GET(HTH_MODA_SEC)/TIEMPO_GET(HTH_MODA_OMP)<<std::endl;
	std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_MODA_SEC)/TIEMPO_GET(HTH_MODA_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_MODA_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_MODA_OMP))<<std::endl;
	std::cout<<"Coste Sec        : nro procesadores*Tiempo             : "<<1<<" * "<<TIEMPO_GET(HTH_MODA_SEC)<<" = "<<TIEMPO_GET(HTH_MODA_SEC)<<std::endl;
	std::cout<<"Coste Omp        : nro procesadores*Tiempo             : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_MODA_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_MODA_OMP)<<std::endl;
	std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(HTH_MODA_OMP)<<" - "<<TIEMPO_GET(HTH_MODA_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_MODA_OMP))-TIEMPO_GET(HTH_MODA_SEC)<<std::endl;


	std::cout<<std::endl;
	std::cout<<"Valores Ideal: "<<std::endl;
	TIEMPO_GET(HTH_MODA_OMP) = TIEMPO_GET(HTH_MODA_SEC) / 2;
	std::cout<<"Tiempo axpy Sec  : "<<TIEMPO_GET(HTH_MODA_SEC)<<" [ms]"<<std::endl;
	std::cout<<"Tiempo axpy Omp  : "<<TIEMPO_GET(HTH_MODA_OMP)<<" [ms]"<<std::endl;

	std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_MODA_SEC)<<" / "<<TIEMPO_GET(HTH_MODA_OMP)<<" = "<<TIEMPO_GET(HTH_MODA_SEC)/TIEMPO_GET(HTH_MODA_OMP)<<std::endl;
	std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_MODA_SEC)/TIEMPO_GET(HTH_MODA_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_MODA_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_MODA_OMP))<<std::endl;
	std::cout<<"Coste Sec        : nro procesadores*Tiempo             : "<<1<<" * "<<TIEMPO_GET(HTH_MODA_SEC)<<" = "<<TIEMPO_GET(HTH_MODA_SEC)<<std::endl;
	std::cout<<"Coste Omp        : nro procesadores*Tiempo             : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_MODA_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_MODA_OMP)<<std::endl;
	std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(HTH_MODA_OMP)<<" - "<<TIEMPO_GET(HTH_MODA_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_MODA_OMP))-TIEMPO_GET(HTH_MODA_SEC)<<std::endl;

}
// ----------------------------------------------------------------------------
"""
text_file = open("code_encuesta.cpp", "w")
text_file.write(code)
text_file.close()

#3.2. Compilación de código C del programa encuesta

In [105]:
!g++ -o encuesta -fopenmp code_encuesta.cpp

#3.3. Ejecución del programa encuesta

In [131]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

cantidad_elementos =   10#@param {type: "number"}
# --------------------------------------------

script_file = f"./encuesta {cantidad_elementos}"
text_file = open("script_file.sh", "w")
text_file.write(script_file)
text_file.close()

%env OMP_NUM_THREADS=2
#!./encuesta 30
!chmod 755 ./script_file.sh
!./script_file.sh
!ls

env: OMP_NUM_THREADS=2
****************************************************************************************
resultados : [9, 7, 0, 9, 1, 6, 9, 6, 0, 6, ]
frecuencias_secuencial : [2, 1, 0, 0, 0, 0, 3, 1, 0, 3, ]
frecuencias_paralelo : [2, 1, 0, 0, 0, 0, 3, 1, 0, 3, ]
sumatoria_secuencial: 53
media_secuencial: 5.3
sumatoria_paralelo: 53
media_paralelo: 5.3
mas_grande_secuencial: 5
valor_moda_secuencial: 6
mas_grande_paralelo: 5
valor_moda_paralelo: 6
****************************************************************************************
Valores Reales  :
Tiempo TOTAL     : 0.271082 [ms]
Tiempo axpy Sec  : 0 [ms]
Tiempo axpy Omp  : 0.102997 [ms]

SpeedUp          : (tiempo Secuencial/tiempo paralelo) : 0 / 0.102997 = 0
Eficiencia       : SpeedUp/nro procesadores            : 0 / 2 = 0
Coste Sec        : nro procesadores*Tiempo             : 1 * 0 = 0
Coste Omp        : nro procesadores*Tiempo             : 2 * 0.102997 = 0.205994
Funcion Overhead : Coste Omp - tiempo Secuencial     

#4. Tabla de pasos

Paso | Procesador | Funcion | Detalle
------------ | ------------ | ------------- | -------------
1 | CPU | @param | Lectura del tamaño de vector de Colab.

#5. Conclusiones

**Medidas de prestaciones en algoritmos paralelos**
Las tecnicas de HPC buscan reducir los tiempos de ejecución, el tiempo como meida, no alcanza. Dos algoritmos pueden ejecutar en el mismo tiempo, pero uno de ellos usa menos procesadores [6,7].

**SpeedUp**
Referencia a la ganacia de velocidad que se consigue con un algoritmo paralelo, al resolver el mismo problema con respecto al algoritmo secuencial.

**Eficiencia**
La eficiencia normaliza el valor del SpeedUp, al dividirlo por la cantidad de procesadores que se utilizaron para alcanzar la ganacia en velocidad. Dando la idea de la porción de tiempo que los procesadores se dedican al trabajo útil.

**Coste**
El coste de un algoritmo paralelo representa el tiempo realizado por todo el sistema en la resoluciòn del problema.

**Función Overhead**
Es la diferencia entre el Coste y el tiempo secuencial. Mientras mayor es la función overhead, peor es el comportamiento del algoritmo paralelo.


#6. Bibliografía
